In [16]:
from datasets import load_dataset
import json
import torch
from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer
import os
from dotenv import load_dotenv

load_dotenv()


True

In [2]:
data_file = "../datasets/gsm8k_step_jepa.jsonl"

In [3]:
dataset = load_dataset('json', data_files=data_file)['train']

Generating train split: 6862 examples [00:00, 220148.35 examples/s]


In [4]:
dataset[0]

{'question': 'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?',
 'answer': 'Natalia sold 48/2 = <<48/2=24>>24 clips in May.\nNatalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.\n#### 72',
 'ground_truth': '72',
 'total_steps': 7,
 'messages': [{'role': 'system',
   'content': 'Please solve the problem step by step (separate steps with double newlines), but keep it short and put your final answer (do not include any other text or units) within \\boxed{}.'},
  {'role': 'user',
   'content': 'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?'},
  {'role': 'assistant',
   'content': "First, I note that Natalia sold 48 clips in April.\n\nIn May, she sold half as many clips as she did in April, which is 24 clips.\n\nTo find the total number of clips sold over bot

In [17]:
# login to huggingface
from huggingface_hub import login
login(token=os.getenv("HF_TOKEN"))


Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [22]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B-Instruct")

In [27]:
formatted_chat = tokenizer.apply_chat_template(
                    dataset[0]['messages'],
                    tokenize=False,
                    add_generation_prompt=True,
                )

In [28]:
print(formatted_chat)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 04 Jan 2026

Please solve the problem step by step (separate steps with double newlines), but keep it short and put your final answer (do not include any other text or units) within \boxed{}.<|eot_id|><|start_header_id|>user<|end_header_id|>

Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

First, I note that Natalia sold 48 clips in April.

In May, she sold half as many clips as she did in April, which is 24 clips.

To find the total number of clips sold over both months, I add the clips sold in April and May: 48 + 24 = 72.

Therefore, Natalia sold a total of 72 clips in April and May.
</think>

**Step 1:** Determine the number of clips Natalia sold in April.
\[
\text{Clips sold in April} = 48
\]

**Step 2:** Calcul

In [26]:
print(formatted_chat)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 04 Jan 2026

Please solve the problem step by step (separate steps with double newlines), but keep it short and put your final answer (do not include any other text or units) within \boxed{}.<|eot_id|><|start_header_id|>user<|end_header_id|>

Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

First, I note that Natalia sold 48 clips in April.

In May, she sold half as many clips as she did in April, which is 24 clips.

To find the total number of clips sold over both months, I add the clips sold in April and May: 48 + 24 = 72.

Therefore, Natalia sold a total of 72 clips in April and May.
</think>

**Step 1:** Determine the number of clips Natalia sold in April.
\[
\text{Clips sold in April} = 48
\]

**Step 2:** Calcul